<font size="5">Build Your Own ANUGA Model (BYOAM) </font>

<font size="3">In this notebook, we will:

- set model configuration parameters

- download and preprocess elevation and landcover datasets

    
- Determine water body polygons


- Build the Digital Elevation Model



</font>

<font size="3">This could take some time, depending on model domain size and complexity of the water mask</font>


<font size=5 color='green'> If you are running in Google Colab, set the variable yes_colab = True. If you are running on your own computer, set the variable yes_colab = False </font>


In [ ]:
yes_colab = False

<font size=5> Step #1: Mount Google Drive and Grant Access <font> <br>




<font size=3> When you run the next cell, a pop-up window will appear asking you to grant access to your Google Drive. You must approve or the notebook will not work. <font> <br>


In [ ]:
import sys
import os
your_path = os.getcwd() + '/'
if yes_colab:
  where_to_mount = '/content/drive/'
  from google.colab import drive
  drive.mount(where_to_mount, force_remount=True)
  mounted_drive = where_to_mount + 'MyDrive/' 
else:
  mounted_drive = your_path

print('Your working directory is %s' %(mounted_drive))

<font size=5> Step #2: Install packages. <font> <br>
<font size=3> This cell should install all Python packages you need for each tutorial.<font>

In [ ]:
import os
os.chdir(mounted_drive)
if yes_colab:
  if os.path.isdir(mounted_drive + 'komo_estuary_tutorial'):
    print('## Updating the local git repository \n')
    os.chdir(mounted_drive + '/komo_estuary_tutorial')
    ! git pull 
  else:
    print('## Pulling the git repository with files for the tutorial\n')
    ! git clone https://github.com/achri19/komo_estuary_tutorial.git

  print('\n## Installing the Python packages needed for these tutorials\n')
  !/bin/bash $mounted_drive/komo_estuary_tutorial/install_packages.sh




In [ ]:
!unzip -n komo_starter.zip -d $mounted_drive 


<font size=5> Step #3: Set up working directories<font> 


<font size=3> We will access data files stored on a shared Google Drive. You will also need to save files to your own Google Drive.<font>



In [ ]:
path_code = mounted_drive + 'processing/code/'
path_templates = mounted_drive + 'processing/templates/'
path_configs = mounted_drive + 'processing/configs/'
path_ancillary = mounted_drive + 'processing/ancillary/'
sys.path.insert(1,path_code)

print(path_code)

<font size=5> Step #4: Now we will import the Python packages we need. <font> 




In [ ]:
import sys
import os
import pandas as pd
import shutil
from datetime import datetime
from string import Template
import fnmatch
import geopandas as gpd
import rasterio
from osgeo import gdal 
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import rtree
import pygeos
import pyTMD


In [ ]:
from BYOM_Utilities_V1 import (build_directory,
                               get_extent_parameters,
                               setup_AOI_files, 
                               make_polygons,
                               make_channel_networks,
                               make_model_foundation, 
                               set_boundary_conditions, 
                               make_watermask,
                               more_opening)

<font size='5' color = 'red' > Building the DEM STEP #1: <br> Set the AOI again and make sure the working directory is set.  </font>


In [ ]:
AOI = 'komo'

Path((mounted_drive + AOI)).mkdir(parents=True, exist_ok=True)

skip = False
res = 30 #meters
print('\n')
print('Study area is ' + AOI)
print('Resolution of this setup is %sm' %(res))

working_path,folders = build_directory(mounted_drive, AOI)
print(working_path)

<font size='5' color = 'red' > Building the DEM STEP #2: <br> Here we set the necessary configuration parameters and then get extent coordinates for the model </font>


In [ ]:
parameters = pd.DataFrame()
parameters['AOI'] = [AOI]
parameters['RiverOceanBoundary'] = '1260'
parameters['Discharge'] = '426'

#Method parameters:
parameters['LandcoverMethod'] = 'WorldCover'
parameters['LandElevMethod'] = 'GLO30'
parameters['OceanElevMethod'] = 'GEBCO'
parameters['LowerRiverElevMethod'] = 'plane'
parameters['UpperRiverElevMethod'] = 'wdpower'
parameters['WetlandElevMethod'] = 'constant_0.5'
parameters['LakeElevMethod'] = 'constant_1'
parameters['ManningLUT'] = 'default'
parameters['WetlandClass'] = '90'

#Coefficients for determining bathymetry:
parameters['WD_POWERA_upper'] = '0.0606'
parameters['WD_POWERB_upper'] = '0.7732'

#Max thresholds:
parameters['MaxOceanDepth'] = '-300'
parameters['MaxNearshoreDepth'] = '-300'
parameters['MaxRiverDepth'] = '-300'
parameters['MaxRiverWidth'] = '756'



In [ ]:
ref_10m,parameters = get_extent_parameters(path_ancillary,AOI,folders,res,parameters)


In [ ]:
plt.imshow(ref_10m.read(1),vmin=-50,vmax=0)
plt.title('GEBCO Bathymetry resampled to 10m resolution')

<font size='5' color = 'red' > The parameters were saved to a configuration file, we open that here </font>


In [ ]:
parameters = pd.read_csv('%s/%s_Configuration.csv' %(folders[2],AOI))
print(parameters.iloc[0] )

<font size='5' color = 'red' > Building the DEM STEP #2b: <br> Download GEBCO, GLO30, World Cover, and Global Mangrove Maps for the area</font>

In [ ]:
ref = setup_AOI_files(your_path,
                    AOI,
                    folders,
                    res,
                    parameters)


<font size='5' color = 'red' > The EPSG coordinate reference system must be is in UTM </font>


In [ ]:
EPSG = parameters['EPSG'][0]
ulx = parameters['ulx'][0]
uly = parameters['uly'][0]
lrx = parameters['lrx'][0]
lry = parameters['lry'][0]


In [ ]:
#ref = rasterio.open('%s/%s_GEBCO_%s.tif' %(folders[8],AOI,res))
glo30 = rasterio.open('%s/%s_GLO30_topo_%s.tif' %(folders[8],AOI,res))
landcover = rasterio.open('%s/%s_WorldCover_%s.tif' %(folders[8],AOI,res))


fig,[ax1,ax2] = plt.subplots(nrows=2,figsize=(10,10))
ax1.imshow(glo30.read(1),vmin=0,vmax=50)
ax1.set_title('Tandem-X GLO30 Topography at 30m resolution')

ax2.imshow(landcover.read(1))
ax2.set_title('WorldCover Landcover Map')


In [ ]:
model_domain = gpd.read_file('%s%s_modeldomain.shp' %(folders[7],AOI))

fig,ax = plt.subplots(figsize=(10,10))
model_domain.geometry.boundary.plot(color=None,edgecolor='red',linewidth = 2,ax=ax,label = 'model domain') #Use your second dataframe

plt.legend()

<font size='5' color = 'red' > Building the DEM STEP #3: <br> Clean, filter, smooth the water mask you made in the previous notebook </font>



In [ ]:
#ref_10m = rasterio.open('%s/%s_GEBCO_10.tif' %(folders[8],AOI))
watermaskname = make_watermask(path_ancillary, 
                               AOI,
                               folders,
                               parameters,
                               ref_10m,False, False)
how_much_opening = 3
more_opening(AOI,folders,watermaskname,how_much_opening,ref_10m,parameters)


In [ ]:
print(watermaskname)
if res != 10:
    os.system('gdalwarp -overwrite -tr %s %s %s/%s_watermask_10.tif %s/%s_watermask_%s.tif '\
                      ' -te %s %s %s %s -srcnodata -9999 -dstnodata -9999 -co COMPRESS=DEFLATE -q'
                      %(res,res,folders[8],AOI,folders[8],AOI,res,ulx,lry,lrx,uly))
    os.system('gdalwarp -overwrite -tr %s %s %s/%s_landmask_10.tif %s/%s_landmask_%s.tif '\
                      ' -te %s %s %s %s -srcnodata -9999 -dstnodata -9999 -co COMPRESS=DEFLATE -q'
                      %(res,res,folders[8],AOI,folders[8],AOI,res,ulx,lry,lrx,uly))


In [ ]:
watermask = rasterio.open('%s%s_watermask_%s.tif' %(folders[8],AOI,res)).read(1)

fig,[ax1,ax2] = plt.subplots(nrows=2,figsize=(15,15))
ax1.imshow(watermask,'gray')

ax2.imshow(watermask,'gray')
ax2.axis([1500,2000, 2000,1500])

<font size='5' color = 'red' > Building the DEM STEP #4: <br> Make polygons of each land cover type: ocean, lake, river, land</font>


In [ ]:
make_polygons(AOI,
                folders,
                parameters,
                ref,
                watermaskname,
                path_templates,False)



In [ ]:
fix,ax = plt.subplots(figsize=(20,20))

colors = ['red','blue','orange','cyan','green']
polys = ['lands','fullocean','lakes','rivers']
i=0
for poly in polys:
    tmp = gpd.read_file([os.path.join(dirpath,f)
            for dirpath,dirnames, files in os.walk(folders[7])
            for f in fnmatch.filter(files,'*%s*.shp' %(poly))][0])
    tmp.geometry.boundary.plot(color=colors[i], edgecolor=colors[i],linewidth = 1,ax=ax,label = poly) #Use your second dataframe
    i=i+1
plt.legend()


In [ ]:
cleanup = False 
if cleanup == True:
    print('Cleaning up temporary files')
    try:shutil.rmtree(folders[1])
    except:''


<font size='5' color = 'red' > Building the DEM STEP #5: <br> Using Orinoco, get distance and width files of the river networks</font>


In [ ]:
segment_width = 150
pixel_step = int(round(segment_width/res))
distance,widths = make_channel_networks(folders,
                                      AOI,
                                      ref,
                                      parameters,
                                      pixel_step,False)


<font size='5' color = 'red' > Building the DEM STEP #7: <br> Make the Digital Elevation Model </font>


In [ ]:
elevation,elev_name = make_model_foundation(mounted_drive,
                                                parameters,
                                                AOI,
                                                folders,
                                                ref,
                                                distance,
                                                widths,
                                                watermask,pixel_step,mounted_drive)


<font size=5 color='green'> We will use the elevation file in later notebooks. </font>

In [ ]:
print(elev_name)


<font size=5 color='red'> Done building DEM and other ancillary files. Move on to the next notebook 3_GetBoundaries.ipynb </font>
